In [1]:
import torch 
import torchvision
import pandas as pd
import numpy as np 

In [2]:
pd.__version__

'0.25.1'

In [3]:
import os
os.chdir("D:\\CAIR, DRDO Internship 2021\\Andrew Ng Machine Learning\\machine-learning-ex8\\ml-latest-small")
rating_df = pd.read_csv("ratings.csv")

In [4]:
rating_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
rating_df.drop("timestamp", axis = 1, inplace = True)

In [6]:
rating_df.head(5)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
len(rating_df.userId.unique())

610

In [8]:
len(rating_df.movieId.unique())

9724

In [9]:
Y = rating_df.pivot(index = "userId",
                   columns = "movieId",
                   values = "rating").reset_index(drop = True)

In [10]:
Y.shape

(610, 9724)

In [11]:
Y.iloc[0:5, 0:15]

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
y = Y.fillna(0)

y.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
f = lambda x:1 if x>0 else 0

R = y.applymap(f)
R.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### User based similarity

In [14]:
### Calculating cosine similarity between users

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

user_sim = 1 - pairwise_distances(y.values, metric ="cosine")

#Storing results in a dataframe
y_df = pd.DataFrame(user_sim)

#Set the index and column names to user ID (0 to 610)
y_df.index = rating_df.userId.unique()
y_df.columns = rating_df.userId.unique()

In [15]:
y_df.iloc[0:5,0:5]

,1,2,3,4,5
1,1.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,1.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,1.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,1.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,1.000000


In [16]:
y_df.shape

(610, 610)

In [17]:
#The diagonal values shows similarity between one user to himself or herself. We need algorithm to find similar user to a specific user. So we will get the diagonal values as 0.0.

In [18]:
np.fill_diagonal(user_sim, 0)
y_df.iloc[0:5,0:5]

## All diagonal values are set to 0, which helps to avoid selecting self as the most similar user

,1,2,3,4,5
1,0.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,0.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,0.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,0.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,0.000000


In [19]:
#Filtering Similar Users

y_df.idxmax(axis=1)[:]

1      266
2      366
3      313
4      391
5      470
      ... 
606    474
607    570
608    480
609    340
610    249
Length: 610, dtype: int64

In [20]:
#Printing the simiarity between user 1 and users ranging from 331 to 340

y_df.iloc[1:2, 360:370]

,361,362,363,364,365,366,367,368,369,370
2,0.012776,0.115081,0.084261,0.0,0.149578,0.300074,0.031699,0.008637,0.016431,0.034816


In [21]:
#Why is there a similarity between user 338 and user 2?

movies_df = pd.read_csv("movies.csv")
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
movies_df.drop( "genres", axis = 1, inplace = True)

In [23]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [24]:
#Finding common movies of similar users

def get_user_similar_movies(user1, user2):
    #inner join between movies watched between two users will give the common movie watched.
    common_movies = rating_df[rating_df.userId == user1].merge(rating_df[rating_df.userId == user2], 
                                                               on = "movieId",
                                                               how = "inner")
    
    #Join the above result set to the movie details
    return common_movies.merge(movies_df, on = "movieId")                                                   

In [25]:
common_movies = get_user_similar_movies(2,366)
common_movies

,userId_x,movieId,rating_x,userId_y,rating_y,title
0,2,3578,4.0,366,4.5,Gladiator (2000)
1,2,6874,4.0,366,4.0,Kill Bill: Vol. 1 (2003)
2,2,48516,4.0,366,4.5,"Departed, The (2006)"
3,2,58559,4.5,366,4.0,"Dark Knight, The (2008)"
4,2,68157,4.5,366,4.5,Inglourious Basterds (2009)
5,2,79132,4.0,366,4.0,Inception (2010)
6,2,91529,3.5,366,4.0,"Dark Knight Rises, The (2012)"
7,2,109487,3.0,366,5.0,Interstellar (2014)
8,2,122882,5.0,366,2.0,Mad Max: Fury Road (2015)


#### Item-based similarity

In [26]:
rating_mat = rating_df.pivot(index = "movieId", columns ="userId", values = "rating").reset_index(drop = True)
rating_mat.head(5)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
1,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Fill all nans with 0

ratings = rating_mat.fillna(0)

In [28]:
#Find the correlation between movies

movies_sim = 1 - pairwise_distances(ratings.values, metric = "correlation")

In [29]:
movies_sim[0:5]

array([[ 1.        ,  0.2313265 ,  0.1732128 , ..., -0.0289057 ,
        -0.0289057 , -0.0289057 ],
       [ 0.2313265 ,  1.        ,  0.19194487, ..., -0.01829073,
        -0.01829073, -0.01829073],
       [ 0.1732128 ,  0.19194487,  1.        , ..., -0.01172923,
        -0.01172923, -0.01172923],
       [-0.02891715,  0.07126889,  0.06714272, ..., -0.0041378 ,
        -0.0041378 , -0.0041378 ],
       [ 0.1924741 ,  0.20052566,  0.37017076, ..., -0.01145552,
        -0.01145552, -0.01145552]])

In [30]:
movies_sim_df = pd.DataFrame(movies_sim)

In [31]:
movies_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,1.000000,0.231327,0.173213,-0.028917,0.192474,0.192686,0.143743,0.085477,0.177245,0.183382,...,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906
1,0.231327,1.000000,0.191945,0.071269,0.200526,0.158341,0.127569,0.141540,-0.021045,0.285086,...,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291
2,0.173213,0.191945,1.000000,0.067143,0.370171,0.196442,0.351513,0.296897,0.275812,0.136916,...,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729
3,-0.028917,0.071269,0.067143,1.000000,0.167910,0.053755,0.258075,0.148726,-0.016025,0.056000,...,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138
4,0.192474,0.200526,0.370171,0.167910,1.000000,0.215503,0.429890,0.265777,0.308085,0.110833,...,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456


In [33]:
movies_sim_df.iloc[0:5,0:5]

,0,1,2,3,4
0,1.000000,0.231327,0.173213,-0.028917,0.192474
1,0.231327,1.000000,0.191945,0.071269,0.200526
2,0.173213,0.191945,1.000000,0.067143,0.370171
3,-0.028917,0.071269,0.067143,1.000000,0.167910
4,0.192474,0.200526,0.370171,0.167910,1.000000


In [35]:
movies_sim_df.shape

(9724, 9724)

#### Finding most similar movies

In [66]:
def get_similar_movies(movieid, topN = 5):
    #Get the index of the movie record in movies_df
    movieidx = movies_df[movies_df.movieId == movieid].index[0]
    movies_df['similarity'] = movies_sim_df.iloc[movieidx]
    top_n = movies_df.sort_values(["similarity"], ascending = False)[0:topN]
    
    return top_n

In [69]:
movies_df[movies_df.movieId == 250]

,movieId,title,similarity
214,250,Heavyweights (Heavy Weights) (1995),-0.009314


In [70]:
get_similar_movies(250)

,movieId,title,similarity
214,250,Heavyweights (Heavy Weights) (1995),1.000000
401,460,Getting Even with Dad (1994),0.742042
294,336,"Walking Dead, The (1995)",0.605819
269,310,Rent-a-Kid (1995),0.605819
218,254,Jefferson in Paris (1995),0.605819
